In [ ]:
import spacy
from spacy.matcher import Matcher
import re
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from joblib import load
from pydantic import BaseModel
from spacy.matcher import Matcher

from datetime import datetime

entities_dict = {}
duration_entities={}



nlp = spacy.load("en_core_web_sm")
query = input("Enter your text:")

app = FastAPI()

origins = [
    "http://localhost",
    "http://localhost:5173",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class Query(BaseModel):
    uid: str
    query: str


def classifyQuery(query):
    
   
    model = load('./model/model.joblib')
    fitted_vectorizer = load('./model/fitted_vectorizer.joblib')
    
    typeId, = model.predict(fitted_vectorizer.transform([query]))
    

    match typeId:
        case 0:
            return "Task"
        
        case 1:
            return "Event"
            
        case 2:
            return "Routine"
            
        case 3:
            return "Project"
        case 4:
            return "Venture"
        
def extractName():
        
    # Define the regular expression pattern
        pattern = r'@\w+(?:\s+\w+)*@'

        # Find all matches of the regular expression in the text input
        matches = re.findall(pattern, query)

        # Loop through the matches and extract the words between the "@" signs
        for match in matches:
            # Remove the "@" signs and any leading or trailing whitespace
            words = match.strip('@').strip()

            # Tokenize the words using Spacy
            words_doc = nlp(words)

        
        
            list=[token.text for token in words_doc]
            output_string = ' '.join(list)
            entities_dict['name'] = output_string

            




def extractEntities(query,typeId):
    
    
    doc = nlp(query)

    start_time = None

    
    if (typeId==0) or(typeId==1) or(typeId==2):

        extractName()


        
        for token in doc:
            # Check if the token is a number
            if token.pos_ == "NUM":
                # Check if the next token is "pm" or "am"
                next_token = doc[token.i+1] if token.i+1 < len(doc) else None
                if not start_time:
                        # This is the start time
                        start_time = token.text + " " + next_token.text
                        time_obj = datetime.strptime(start_time, '%I.%M %p')
                        time_24_str = time_obj.strftime('%H:%M')
                        entities_dict['s'] = time_24_str

                       # print("Start time:", start_time)
                        
                        
                elif next_token and (next_token.text == "hours"):
                    # The number is part of a duration expression
                    duration_hours = str(token.text)
                    entities_dict['duration']=duration_hours+" hours"
                    duration_entities['h']=duration_hours
                    duration_entities['m']=0
                    entities_dict['duration']=duration_entities
                    

                    
                
                # print("Duration:", duration, "hours")

                elif next_token and (next_token.text == "minutes"):
                    duration_minutes=str(token.text)
                    duration_entities['h']=duration_hours
                    duration_entities['m']=duration_minutes
                    entities_dict['duration']=duration_entities
                    
                    
                    
                    

                else:
                    end_time=token.text+ " " + next_token.text
                    entities_dict['end_time'] = end_time
                    #print("End time:", end_time)

    
        print(entities_dict)


    if (typeId==3): 

        extractName()
        # Extract date
        for ent in doc.ents:
            if ent.label_ == "DATE":
                date_entity=ent.text
                #print(ent.text)
                entities_dict['due']=date_entity
        print(entities_dict)


    if (typeId==4):
        extractName()


        print(entities_dict)



        
        


     
    
classifyQuery(query)
typeId=classifyQuery(query)
extractEntities(query,typeId)


